# Prerequisites
- dwgsim https://github.com/nh13/DWGSIM
- samtools https://github.com/samtools/samtools

##### You can install easily via anaconda 
```bash
conda install -y dwgsim samtools bcftools bwa
```

### This notebook shows how to create In Silico (computational) mutations to give training and testing examples for DeepVCF. We keep track of the mutations introduced via Variant Format Files (VCF). It's not important if the mutatations are found in the wild, but rather to demonstrate how a deep learning model will train on these mutations and determine variant calls. If you would like to see DeepVCF use a high-confidence provided from Genome in a Bottle [GIAB]() then please see the notebook [giab notebook](./giab)

In [1]:
from DeepVCF import pathing
from DeepVCF import pandas as pd
from DeepVCF import alignment
from DeepVCF import dwgsim

=== No GPU Detected ===


/home/tmsincomb/anaconda3/envs/samtools/lib/python3.9/site-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:381: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


# Create Reads With In Silco Mutations 

In [4]:
dwgsim(
    ref_file='./example_files/staphylococcus/train/MRSA107.fasta',
    output_folder='./example_files/staphylococcus/train/',
    output_prefix='train-base-error-10percent',
    **{
        '-r': .005,  # mutation rate :: need to get about 20K mutations for usable training,
        '-e': '.1-.1',  # 10-10% error rate forward
        '-E': '.1-.1',  # 10-10% error rare backward
        '-C': 50,  # mean coverage :: to reduce runtime since 50 is plenty for the tensor
        '-1': 150, # read length
        '-2': 150, # read length
    }
)

dwgsim -r 0.005 -e .1-.1 -E .1-.1 -C 50 -1 150 -2 150 /home/tmsincomb/Dropbox/git/DeepVCF/jupyter_nb/example_files/staphylococcus/train/MRSA107.fasta train-base-error-10percent


# Variant Format File (VCF) With In Silco Mutations 

In [6]:
vcf = pd.read_vcf('./example_files/staphylococcus/train/train-base-error-10percent.mutations.vcf')
print('Mutations Introduced:', vcf.shape[0])

Mutations Introduced: 15503


In [7]:
vcf.head()

,CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT,Sample1
0,CP018629.1,347,.,T,G,100,PASS,AF=0.5;pl=1;mt=SUBSTITUTE,GT,{'GT': '0/1'}
1,CP018629.1,393,.,T,C,100,PASS,AF=1.0;pl=3;mt=SUBSTITUTE,GT,{'GT': '1/1'}
2,CP018629.1,618,.,G,C,100,PASS,AF=0.5;pl=1;mt=SUBSTITUTE,GT,{'GT': '0/1'}
3,CP018629.1,767,.,A,G,100,PASS,AF=0.5;pl=1;mt=SUBSTITUTE,GT,{'GT': '0/1'}
4,CP018629.1,795,.,C,T,100,PASS,AF=0.5;pl=2;mt=SUBSTITUTE,GT,{'GT': '0/1'}


# Alignment Back to the Genome with the introduced mutations

In [8]:
pysam_obj = alignment.bwa_mem(
    reference='./example_files/staphylococcus/train/MRSA107.fasta',
    reads=[
        './example_files/staphylococcus/train/train-base-error-10percent.bwa.read1.fastq', 
        './example_files/staphylococcus/train/train-base-error-10percent.bwa.read2.fastq'
    ],
    output_folder='./example_files/staphylococcus/train/',
    output_prefix='train-base-error-10percent',
)

bwa index /home/tmsincomb/Dropbox/git/DeepVCF/jupyter_nb/example_files/staphylococcus/train/MRSA107.fasta
bwa mem -t 6 /home/tmsincomb/Dropbox/git/DeepVCF/jupyter_nb/example_files/staphylococcus/train/MRSA107.fasta /home/tmsincomb/Dropbox/git/DeepVCF/jupyter_nb/example_files/staphylococcus/train/train-base-error-10percent.bwa.read1.fastq /home/tmsincomb/Dropbox/git/DeepVCF/jupyter_nb/example_files/staphylococcus/train/train-base-error-10percent.bwa.read2.fastq > /home/tmsincomb/Dropbox/git/DeepVCF/jupyter_nb/example_files/staphylococcus/train/train-base-error-10percent.sam
samtools view -Shb /home/tmsincomb/Dropbox/git/DeepVCF/jupyter_nb/example_files/staphylococcus/train/train-base-error-10percent.sam > /home/tmsincomb/Dropbox/git/DeepVCF/jupyter_nb/example_files/staphylococcus/train/train-base-error-10percent.unsorted.bam
samtools sort -n /home/tmsincomb/Dropbox/git/DeepVCF/jupyter_nb/example_files/staphylococcus/train/train-base-error-10percent.unsorted.bam -o /home/tmsincomb/Dropbo

# Sequence Alignment Mapping (SAM/BAM/CRAM) to be used for DeepVCFs tensor input

In [9]:
bam = pd.read_seq('./example_files/staphylococcus/train/train-base-error-10percent.mapped.bam', format='bam')
bam.head()

,name,flag,ref_name,ref_pos,map_quality,cigar,next_ref_name,next_ref_pos,length,seq,qual,tags
0,CP018629.1_381_4_1_0_0_0_13:1:0_14:0:0_74430,163,CP018629.1,4,60,119M31S,=,381,527,TTACTCCATCTTTTTTAAATTTTTAAGTTATGCACAAAAATAACAA...,")+)',-*+**+--+*,)+,.(,++++,,.,*'+),*-++,(+++2-...","[NM:i:6, MD:Z:13C17A33A6G25C1A18, AS:i:89, XS:..."
1,CP018629.1_14_286_0_1_0_0_15:0:0_20:1:0_17d4,99,CP018629.1,14,60,150M,=,286,422,TTTCTTAAATTTTTAAGTTATACACAAAAATAACGACCGAGGATAA...,"*-+++-0+*+,*+,'*)+++-.,-,)-**+,,,++-+-,*,,+,++...","[NM:i:15, MD:Z:34A4T15A14T10T5A1A3G12A0T4C13A3..."
2,CP018629.1_320_21_1_0_0_0_19:2:0_11:0:0_7cf4c,163,CP018629.1,21,60,150M,=,320,449,AACTTTTAAGTTATACACAAAAATAACAACCGTGGATAACGTCTAA...,"+**&.*,++++,-)++*.,++++,+-*++,.-++*/,+-)++-++,...","[NM:i:11, MD:Z:2T37T7A2C2T8T14T6C1G5A40A15, AS..."
3,CP018629.1_24_403_0_1_0_0_12:0:0_16:0:0_7c5f2,99,CP018629.1,24,60,150M,=,403,517,TTTTAAGTTATACACAAAGATGACAACCGTGGATAATATCTAAATA...,",++-+*++++,'++++++,-)+++++)+/)+)-),++,*-,*++/+...","[NM:i:12, MD:Z:18A2A14C0T6C1C4T12C26A23C2A6T24..."
4,CP018629.1_27_425_0_1_0_0_10:0:0_15:0:0_715bb,99,CP018629.1,27,60,150M,=,425,548,TAAGTTATACACAAAAATAACAACCGTAGATACCATCTAAACACAC...,"-)+-+)),-**.,*,+-,+-+,,+)+*+,+++++*,)-++,+/*++...","[NM:i:10, MD:Z:27G4A1T16T13A17G18A2A7A5T30, AS..."


# Create Testing Data :: Verbose = False to show silent option

In [10]:
# Quietly create testing
prefix='test-base-error-10percent'
folder_path=pathing('./example_files/staphylococcus/test')

dwgsim(
    ref_file=folder_path / 'GCF_000013425.1_ASM1342v1_genomic_reference.fasta',
    output_folder=folder_path,
    output_prefix=prefix,
    verbose=False,
    **{
        '-r': .005,  # mutation rate :: need to get about 20K mutations for usable training,
        '-e': '.1-.1',  # 10-10% error rate forward :: for attempt of PacBio
        '-E': '.1-.1',  # 10-10% error rare backward
        '-C': 50,  # mean coverage
        '-1': 150, # read length
        '-2': 150, # read length
    }
)
_ = alignment.bwa_mem(
    reference=folder_path / 'GCF_000013425.1_ASM1342v1_genomic_reference.fasta',
    reads=[
        folder_path / f'{prefix}.bwa.read1.fastq', 
        folder_path / f'{prefix}.bwa.read2.fastq'
    ],
    output_folder=folder_path,
    output_prefix=prefix,
    verbose=False,
)

# Create a comparison test with default 2% platform base error rate

In [11]:
### DEFAULT TRAIN ###
prefix='train-base-error-2percent'
folder_path=pathing('./example_files/staphylococcus/train')

dwgsim(
    ref_file=folder_path / 'MRSA107.fasta',
    output_folder=folder_path,
    output_prefix=prefix,
    verbose=False,
    **{
        '-r': .005,  # mutation rate :: need to get about 20K mutations for usable training,
        '-C': 50,  # mean coverage
        '-1': 150, # read length
        '-2': 150, # read length
    }
)
_ = alignment.bwa_mem(
    reference=folder_path / 'MRSA107.fasta',
    reads=[
        folder_path / f'{prefix}.bwa.read1.fastq', 
        folder_path / f'{prefix}.bwa.read2.fastq'
    ],
    output_folder=folder_path,
    output_prefix=prefix,
    verbose=False,
)

### DEFAULT TEST ###
prefix='test-base-error-2percent'
folder_path=pathing('./example_files/staphylococcus/test')

dwgsim(
    ref_file=folder_path / 'GCF_000013425.1_ASM1342v1_genomic_reference.fasta',
    output_folder=folder_path,
    output_prefix=prefix,
    verbose=False,
    **{
        '-r': .005,  # mutation rate :: need to get about 20K mutations for usable training,
        '-e': '.1-.1',  # 10-10% error rate forward :: for attempt of PacBio
        '-E': '.1-.1',  # 10-10% error rare backward
        '-C': 50,  # mean coverage
        '-1': 150, # read length
        '-2': 150, # read length
    }
)
_ = alignment.bwa_mem(
    reference=folder_path / 'GCF_000013425.1_ASM1342v1_genomic_reference.fasta',
    reads=[
        folder_path / f'{prefix}.bwa.read1.fastq', 
        folder_path / f'{prefix}.bwa.read2.fastq'
    ],
    output_folder=folder_path,
    output_prefix=prefix,
    verbose=False,
)

# We Now want to compare out variant calls to the Samtools/BCFTools variant caller.

In [1]:
from DeepVCF import bcftools_vcf
samtools_variant_calls_be2percent = bcftools_vcf(
    ref_file='./example_files/staphylococcus/test/GCF_000013425.1_ASM1342v1_genomic_reference.fasta', 
    bam_file='./example_files/staphylococcus/test/test-base-error-2percent.mapped.bam',
    output_folder='./example_files/staphylococcus/',
    output_prefix='GCF_000013425-base-error-2percent'
)
samtools_variant_calls_be10percent = bcftools_vcf(
    ref_file='./example_files/staphylococcus/test/GCF_000013425.1_ASM1342v1_genomic_reference.fasta', 
    bam_file='./example_files/staphylococcus/test/test-base-error-10percent.mapped.bam',
    output_folder='./example_files/staphylococcus/',
    output_prefix='GCF_000013425-base-error-10percent'
)

=== No GPU Detected ===


/home/tmsincomb/anaconda3/envs/samtools/lib/python3.9/site-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:381: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


# Our Data is ready!

In [16]:
ref_train_be10percent = pathing('./example_files/staphylococcus/train/MRSA107.fasta')
bam_train_be10percent = pathing('./example_files/staphylococcus/train/train-base-error-10percent.mapped.bam')
vcf_train_be10percent = pathing('./example_files/staphylococcus/train/train-base-error-10percent.mutations.vcf')

ref_test_be10percent = pathing('./example_files/staphylococcus/test/GCF_000013425.1_ASM1342v1_genomic_reference.fasta')
bam_test_be10percent = pathing('./example_files/staphylococcus/test/test-base-error-10percent.mapped.bam')
vcf_test_be10percent = pathing('./example_files/staphylococcus/test/test-base-error-10percent.mutations.vcf')

ref_train_be2percent = pathing('./example_files/staphylococcus/train/MRSA107.fasta')
bam_train_be2percent = pathing('./example_files/staphylococcus/train/train-base-error-2percent.mapped.bam')
vcf_train_be2percent = pathing('./example_files/staphylococcus/train/train-base-error-2percent.mutations.vcf')

ref_test_be2percent = pathing('./example_files/staphylococcus/test/GCF_000013425.1_ASM1342v1_genomic_reference.fasta')
bam_test_be2percent = pathing('./example_files/staphylococcus/test/test-base-error-2percent.mapped.bam')
vcf_test_be2percent = pathing('./example_files/staphylococcus/test/test-base-error-2percent.mutations.vcf')

samtools_variant_calls_be2percent = pathing('./example_files/staphylococcus/GCF_000013425-base-error-2percent.bcftools.vcf')
samtools_variant_calls_be10percent = pathing('./example_files/staphylococcus/GCF_000013425-base-error-10percent.bcftools.vcf')